In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection as ms
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import precision_recall_fscore_support as sk
from imblearn.over_sampling import SMOTE

In [2]:
#먼저 데이터 읽어오기
df=pd.read_excel('../optimal_data3_1/'+'Continous_2weeks_2day_1term.xlsx')
df.head()
X=df.iloc[:,[1,3,4,5,6,7]]
y=df.iloc[:,-1]

#스케일링 먼저 하고 ndarray를 dataframe으로 변환
scaler = StandardScaler()
X=scaler.fit_transform(X)
X=pd.DataFrame(X)

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]

In [3]:
#한 사람당 데이터 수
Count_1=int(181*0.1)
#한 사람당 데이터 수
Count_2=181

X_test=pd.DataFrame()
X_train=pd.DataFrame()
y_test=pd.DataFrame()
y_train=pd.DataFrame()
empty=pd.DataFrame()

for i in range(10):
    X_test=empty
    X_train=empty
    y_test=empty
    y_train=empty
    for j in range(322):
        X_temp_test=X.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        X_test=pd.concat([X_test,X_temp_test])
        X_temp_train=X.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        X_train=pd.concat([X_train,X_temp_train])
        
        y_temp_test=y.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        y_test=pd.concat([y_test,y_temp_test])
        y_temp_train=y.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        y_train=pd.concat([y_train,y_temp_train])
    
    #SMOTE 적용
    smote = SMOTE(random_state=0)
    X_train,y_train = smote.fit_resample(X_train,y_train)
    X_test,y_test = smote.fit_resample(X_test,y_test)
    
    #모델 정의, 예측
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    
    predict=model.predict(X_test)
    #Accuracy
    print("[{}]Accuracy : {}".format(i,model.score(X_test,y_test)))
    #f1score
    f1 = f1_score(y_test,predict, average='weighted')
    print("[{}]F1score : {}".format(i,f1))
    #precision/recall
    list=sk(y_test,predict,average='weighted')
    print("[{}]Precision : {}".format(i,list[0]))
    print("[{}]Recall : {}".format(i,list[1]))
    print()
    
    #결과 배열에 넣기
    Result[i][0]=model.score(X_test,y_test)
    Result[i][1]=list[2]
    Result[i][2]=list[0]
    Result[i][3]=list[1]
    print("{} : {}".format(list[2],f1))

[0]Accuracy : 0.4919535808168237
[0]F1score : 0.4738058671559188
[0]Precision : 0.4789343809242837
[0]Recall : 0.4919535808168237

0.4738058671559188 : 0.4738058671559188
[1]Accuracy : 0.8890181630146726
[1]F1score : 0.8873454498422463
[1]Precision : 0.8893968864451648
[1]Recall : 0.8890181630146726

0.8873454498422463 : 0.8873454498422463
[2]Accuracy : 0.8754284289898423
[2]F1score : 0.8732200884175455
[2]Precision : 0.8768949762574864
[2]Recall : 0.8754284289898423

0.8732200884175455 : 0.8732200884175455
[3]Accuracy : 0.8454455567914967
[3]F1score : 0.8416864751817617
[3]Precision : 0.8458455490947745
[3]Recall : 0.8454455567914967

0.8416864751817617 : 0.8416864751817617
[4]Accuracy : 0.8680479159027014
[4]F1score : 0.8655434476741917
[4]Precision : 0.8688841395501287
[4]Recall : 0.8680479159027014

0.8655434476741917 : 0.8655434476741917
[5]Accuracy : 0.7642836128273022
[5]F1score : 0.7530876877347681
[5]Precision : 0.7672907484999744
[5]Recall : 0.7642836128273022

0.753087687734

In [4]:
Result_df=pd.DataFrame(Result,columns=['Accuracy','F1-Score','Precision','Recall'])
Result_df

,Accuracy,F1-Score,Precision,Recall
0,0.491954,0.473806,0.478934,0.491954
1,0.889018,0.887345,0.889397,0.889018
2,0.875428,0.873220,0.876895,0.875428
3,0.845446,0.841686,0.845846,0.845446
4,0.868048,0.865543,0.868884,0.868048
5,0.764284,0.753088,0.767291,0.764284
6,0.842379,0.838873,0.843942,0.842379
7,0.826277,0.821679,0.825219,0.826277
8,0.840809,0.837024,0.839695,0.840809
9,0.846217,0.842075,0.847458,0.846217


In [5]:
print("Average of Accuracy {}".format(Result_df['Accuracy'].mean()))
print("Average of F1-Score {}".format(Result_df['F1-Score'].mean()))
print("Average of Precision {}".format(Result_df['Precision'].mean()))
print("Average of Recall {}".format(Result_df['Recall'].mean()))

Average of Accuracy 0.8089859654253292
Average of F1-Score 0.8034339394453374
Average of Precision 0.8083561300683122
Average of Recall 0.8089859654253292


In [6]:
Matrix=pd.DataFrame(Result_df['Accuracy'],columns=['Accuracy'])
Matrix['Accuracy']=Result_df['Accuracy']
A=[Result_df['Accuracy'].mean(),Result_df['F1-Score'].mean(),Result_df['Precision'].mean(),Result_df['Recall'].mean()]
A=pd.DataFrame(A,columns=['Accuracy'])
Matrix=pd.concat([Matrix,A])
Matrix=Matrix.transpose()
Matrix.to_excel('./PFMatrix2.xlsx')